# Introduction

In this section, I'll be performing some exploratory data analysis on the data on the Space X Launches that I scraped from Wikipedia. The data analysis will be done with SQL this time. To use SQL to query the data, I will connect to a DB file I created and then load the CSV file into the database as a new table using Pandas.

## Setting Up Environment

In [1]:
%load_ext sql

In [2]:
import sqlite3
import csv
import pandas as pd
import numpy as np
import sqlalchemy

con = sqlite3.connect("datasets/SpaceXDB.db")
cur = con.cursor()

In [3]:
%sql sqlite:///datasets/SpaceXDB.db

In [4]:
data = 'datasets/launch_data_falcon9_wiki.csv'
df = pd.read_csv(data)

# df['Payload mass'] = pd.to_numeric(df['Payload mass'])  # change mass from text to number
# df['Datetime'] = pd.to_datetime(df['Datetime'], dayfirst=True)  # change datetime to datetime format

# load the dataframe into SpaceXDB
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

121

Let's look at the column data types quick:

In [5]:
%%sql
PRAGMA table_info(SPACEXTBL)

 * sqlite:///datasets/SpaceXDB.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Flight_Number,INTEGER,0,None,0
1,Launch_Site,TEXT,0,None,0
2,Payload,TEXT,0,None,0
3,Payload_Mass,REAL,0,None,0
4,Orbit,TEXT,0,None,0
5,Customer,TEXT,0,None,0
6,Mission_Outcome,TEXT,0,None,0
7,Booster_Version,TEXT,0,None,0
8,Landing_Outcome,TEXT,0,None,0
9,Datetime,TEXT,0,None,0


## Querying the Database with SQL

Now the database is set up and the table has been loaded in.

### Task 1

##### Display the names of the unique launch sites  in the space mission.

In [6]:
%%sql
SELECT DISTINCT Launch_Site
FROM SPACEXTBL;

 * sqlite:///datasets/SpaceXDB.db
Done.


Launch_Site
CCSFS SLC-40
VAFB SLC-4E
KSC LC-39A


### Task 2

##### Display 5 records where launch sites begin with the string 'CCA'

Cape Canaveral Space Force Station used to be called Cape Canaveral Air Force Station. I changed all of them to be "Space Force Station" for consistency.

In [7]:
%%sql
SELECT * 
FROM SPACEXTBL
WHERE Launch_Site
LIKE "CCS%"
LIMIT 5;

 * sqlite:///datasets/SpaceXDB.db
Done.


Flight_Number,Launch_Site,Payload,Payload_Mass,Orbit,Customer,Mission_Outcome,Booster_Version,Landing_Outcome,Datetime
1,CCSFS SLC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,F9 v1.0 B0003.1,Failure (parachute),2010-06-04 18:45:00
2,CCSFS SLC-40,Dragon,0.0,LEO,NASA (COTS) NRO,Success,F9 v1.0 B0004.1,Failure (parachute),2010-12-08 15:43:00
3,CCSFS SLC-40,Dragon,525.0,LEO,NASA (COTS),Success,F9 v1.0 B0005.1,No attempt,2012-05-22 07:44:00
4,CCSFS SLC-40,SpaceX CRS-1,4700.0,LEO,NASA (CRS),Success,F9 v1.0 B0006.1,No attempt,2012-10-08 00:35:00
5,CCSFS SLC-40,SpaceX CRS-2,4877.0,LEO,NASA (CRS),Success,F9 v1.0 B0007.1,No attempt,2013-03-01 15:10:00


### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [8]:
%%sql
SELECT Customer, SUM(Payload_Mass) as Total_Payload_Carried
FROM SPACEXTBL
GROUP BY Customer
HAVING Customer='NASA (CRS)';

 * sqlite:///datasets/SpaceXDB.db
Done.


Customer,Total_Payload_Carried
NASA (CRS),59941.0


### Task 4

##### Display average payload mass carried by booster version F9 v1.1

In [9]:
%%sql
WITH Boosters AS (
    SELECT Booster_Version, Payload_Mass
    FROM SPACEXTBL 
    WHERE Booster_Version LIKE 'F9 v1.1%'
)

SELECT AVG(Payload_Mass) as Avg_Payload_Mass
FROM Boosters;

 * sqlite:///datasets/SpaceXDB.db
Done.


Avg_Payload_Mass
2534.6666666666665


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.

*Hint:Use min function*

In [10]:
%%sql
SELECT MIN(Datetime) AS First_Success_Ground_Pad
FROM SPACEXTBL
WHERE Landing_Outcome = "Success (ground pad)";

 * sqlite:///datasets/SpaceXDB.db
Done.


First_Success_Ground_Pad
2015-12-22 01:29:00


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [11]:
%%sql
SELECT Booster_Version, Launch_Site, Payload_Mass, Landing_Outcome
FROM SPACEXTBL
WHERE Landing_Outcome = "Success (drone ship)" AND (Payload_Mass > 4000 AND Payload_Mass < 6000);

 * sqlite:///datasets/SpaceXDB.db
Done.


Booster_Version,Launch_Site,Payload_Mass,Landing_Outcome
F9 FT B1022.1,CCSFS SLC-40,4696.0,Success (drone ship)
F9 FT B1026.1,CCSFS SLC-40,4600.0,Success (drone ship)
F9 FT B1021.2,KSC LC-39A,5300.0,Success (drone ship)
F9 FT B1031.2,KSC LC-39A,5200.0,Success (drone ship)
F9 B5 B1046.2,CCSFS SLC-40,5800.0,Success (drone ship)
F9 B5 B1047.2,KSC LC-39A,5300.0,Success (drone ship)
F9 B5 B1048.3,CCSFS SLC-40,4850.0,Success (drone ship)
F9 B5 B1060.1,CCSFS SLC-40,4311.0,Success (drone ship)
F9 B5 B1058.2,CCSFS SLC-40,5500.0,Success (drone ship)
F9 B5 B1062.1,CCSFS SLC-40,4311.0,Success (drone ship)


### Task 7

##### List the total number of successful and failure mission outcomes

In [12]:
%%sql
SELECT Mission_Outcome, COUNT(*) AS "Count"
FROM SPACEXTBL
GROUP BY Mission_Outcome;

 * sqlite:///datasets/SpaceXDB.db
Done.


Mission_Outcome,Count
Failure,1
Success,120


### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [13]:
%%sql
SELECT DISTINCT Booster_Version, Payload_Mass
FROM SPACEXTBL
WHERE Payload_Mass = (SELECT MAX(Payload_Mass) FROM SPACEXTBL);

 * sqlite:///datasets/SpaceXDB.db
Done.


Booster_Version,Payload_Mass
F9 B5 B1048.4,15600.0
F9 B5 B1049.4,15600.0
F9 B5 B1051.3,15600.0
F9 B5 B1056.4,15600.0
F9 B5 B1048.5,15600.0
F9 B5 B1051.4,15600.0
F9 B5 B1049.5,15600.0
F9 B5 B1060.2,15600.0
F9 B5 B1058.3,15600.0
F9 B5 B1051.6,15600.0


### Task 9

##### List the records which will display the month names, failure landing_outcomes in drone ship, booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6, 2) as month to get the months and substr(Date,1,4)='2015' for year.**

In [14]:
%%sql
SELECT substr(Datetime, 6, 2) AS "Month", substr(Datetime,1,4) AS "Year", Landing_Outcome, Booster_Version, Launch_Site 
FROM SPACEXTBL
WHERE substr(Datetime,1,4)='2015' AND Landing_Outcome LIKE "Failure (drone ship)";

 * sqlite:///datasets/SpaceXDB.db
Done.


Month,Year,Landing_Outcome,Booster_Version,Launch_Site
01,2015,Failure (drone ship),F9 v1.1 B1012,CCSFS SLC-40
04,2015,Failure (drone ship),F9 v1.1 B1015,CCSFS SLC-40


### Task 10

##### Rank the  count of  successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order.

In [15]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS "COUNT"
FROM SPACEXTBL
GROUP BY Landing_Outcome
HAVING Datetime > '2010-04-06' AND Datetime < '2017-03-20'
ORDER BY "COUNT" DESC; 

 * sqlite:///datasets/SpaceXDB.db
Done.


Landing_Outcome,COUNT
Success (drone ship),64
No attempt,22
Success (ground pad),16
Failure (drone ship),8
Controlled (ocean),5
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
